In [1]:
# libs
import pandas as pd
import json
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

dados = pd.read_csv('DB_SHOW_APPLE.csv')
dados.head(2)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO
0,58541000170,Q COISA BOA BAR LTDA,BUTECO TOMAUMA,RUA MATEUS LEME,2631,NaN,BOM RETIRO,CURITIBA,PR,80520174.0,(41)98713461,DELIMA.ADV@GMAIL.COM,5611205,Restaurantes e similares
1,108202000151,ARMAZEM COQUEIRO LTDA,ARMAZEM COQUEIRO,ESTRADA DO CABRAL,0,NaN,CABRAL,PARATY,RJ,23970000.0,(24)337127,VITRINA@VITRINACONTABILIDADE.COM.BR,5611205,Restaurantes e similares


In [2]:
dados['ENDERECO'] = dados['BAIRRO'].map(str) + ", " + dados['CIDADE'].map(str) + ", " + dados['ESTADO'].map(str)
dados['COORDENADAS'] = dados['ENDERECO'].apply(lambda x: geolocator.geocode(x, timeout=10))
dados['COORDENADAS']

0       (Bom Retiro, Matinhos, Região Geográfica Imedi...
1       (Rua Pedro Álvares Cabral, Paraty, Região Geog...
2       (Centro, Nova Veneza, Região Geográfica Imedia...
3       (Centro Comercial Bom Princípio, 430, ERS-122,...
4       (Três Barras, Linhares, Região Geográfica Imed...
                              ...                        
2495    (Rincão do Cascalho, Portão, Região Geográfica...
2496    (Granja Votorantim, Ibiúna, Região Imediata de...
2497                                                 None
2498    (Vila São Paulo, São Dimas, Colombo, Região Ge...
2499    (Pirajá, Salvador, Região Geográfica Imediata ...
Name: COORDENADAS, Length: 2500, dtype: object

In [3]:

dados['LATITUDE'] = dados.apply(lambda row: row['COORDENADAS'].latitude if row['COORDENADAS'] is not None else None, axis=1)
dados['LONGITUDE']  = dados.apply(lambda row: row['COORDENADAS'].longitude if row['COORDENADAS'] is not None else None, axis=1)


In [4]:
dados = dados.sort_values(by='TELEFONE', ascending=True)

In [5]:
import random
horario = ["""domingo 
Fechado

segunda-feira
11:00–15:00
18:00–23:00

terça-feira
11:00–15:00
18:00–23:00

quarta-feira
11:00–15:00
18:00–23:00

quinta-feira
11:00–15:00
18:00–23:00

sexta-feira
11:00–15:00
18:00–23:00

sábado
11:00–15:00
18:00–23:00""",
"""domingo
Fechado

segunda-feira
11:00–15:00

terça-feira
11:00–15:00

quarta-feira
11:00–15:00

quinta-feira
11:00–15:00

sexta-feira
11:00–15:00

sábado
Fechado""",
"""
domingo
Fechado

segunda-feira
10:00–14:30
18:00–00:00

terça-feira
10:00–14:30
18:00–00:00

quarta-feira
10:00–14:30
18:00–00:00

quinta-feira
10:00–14:30
18:00–00:00

sexta-feira
10:00–14:30
18:00–00:00

sábado
10:00–14:30
18:00–00:00""",
"""domingo
Fechado

segunda-feira
11:00–15:30

terça-feira
11:00–15:30

quarta-feira
11:00–15:30

quinta-feira
11:00–15:30

sexta-feira
11:00–15:30

sábado
11:00–15:00"""]
import time
for cnpj in range(0,len(dados['CNPJ'])-1 ):
    dados.at[cnpj, 'HORARIO_FUNCIONAMENTO'] = horario[random.randint(0, 3)]
dados.head(2)



,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO,ENDERECO,COORDENADAS,LATITUDE,LONGITUDE,HORARIO_FUNCIONAMENTO
199,4941979000153,L' ABBATE COMERCIAL LTDA,Indisponível,RUA WISARD,397,LOJA 02,VILA MADALENA,SAO PAULO,SP,5434080.0,(11),EDUARDO.SIKLER@CONTABILCONSULT.COM.BR,5611205,Restaurantes e similares,"VILA MADALENA, SAO PAULO, SP","(Vila Madalena, Rua Heitor Penteado, Sumarezin...",-23.546496,-46.691124,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...
158,4260571000116,AKSA GOURMET LOUNGE E RESTAURANTE EIRELI,AKSA GOURMET LOUNGE,AVENIDA GUILHERME GIORGI,1050,NaN,VILA CARRAO,SAO PAULO,SP,3422001.0,(11)045654,MMHAGE2019@GMAIL.COM,5611205,Restaurantes e similares,"VILA CARRAO, SAO PAULO, SP","(Vila Carrão, Carrão, São Paulo, Região Imedia...",-23.549867,-46.538580,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...


In [6]:
dados['SITE'] = "www" + dados['NOME_FANTASIA'].str.replace(" ", "", regex=True) + ".COM"
dados['FACEBOOK'] = "https://pt-br.facebook.com/" + dados['NOME_FANTASIA'].str.lower().replace(" ","",regex=True)
dados['INSTAGRAM'] = "@"+ dados['NOME_FANTASIA'].str.lower().replace(" ","", regex=True)
opcoes_servicos = [
    """
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """,
    """
Para viagem
Refeição no local
    """,
    """
Mesas externas
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """,
    """
Retirada na porta
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """
]
import time
aleatorio = random.randint(0, 3)
for cnpj in range(0,len(dados['CNPJ'])-1 ):
    dados.at[cnpj, 'OPCOES_DE_SERVICO'] = opcoes_servicos[random.randint(0, 3)]
    #time.sleep(0.05)
dados['OPCOES_DE_SERVICO'].head(2)

199    \nMesas externas\nEntrega sem contato\nEntrega...
158    \nMesas externas\nEntrega sem contato\nEntrega...
Name: OPCOES_DE_SERVICO, dtype: object

In [7]:
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE',
       'CNAE_DESCRICAO', 'HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO']]


In [8]:
indice_remove = dados[(dados['TELEFONE'] =="(nan)nan")].index
dados.drop(indice_remove, inplace=True)
len(dados)


1099

In [9]:
data_teste = dados.copy()
data = dados.copy()

data.rename(columns={'CNPJ':"EIN (CNPJ)", 'RAZAO_SOCIAL':'CORPORATE_NAME', 'NOME_FANTASIA':'TRADING NAME', 'RUA':'STREET', 'NUMERO':'ADDRESS_NUMBER', 'COMPLEMENTO':'ADDRESS_COMPLEMENT',
       'BAIRRO':'DISTRICT', 'CIDADE':'CITY', 'ESTADO':'STATE', 'CEP':'ZIP_CODE', 'TELEFONE':'PHONE_NUMBER', 'EMAIL':'E-MAIL_ADDRESS','HORARIO_FUNCIONAMENTO':'OPENING_HOURS', 'OPCOES_DE_SERVICO':'SERVICE_OPTIONS', 'CNAE_DESCRICAO':'CATEGORY_(CNAE)'}, inplace=True)


In [10]:
# English version
data_teste.to_json("en_amostra_json/EN_db_SIGA_05_APPLE.json", orient="records")
data_teste['OPENING_HOURS','SERVICE_OPTIONS','INSTAGRAM','FACEBOOK'] = None
data_teste.to_json("en_amostra_json/EN_db_SIGA_04_APPLE.json", orient="records")
data_teste['CATEGORY_(CNAE)']  = None
data_teste.to_json("en_amostra_json/EN_db_SIGA_03_APPLE.json", orient="records")
data_teste['SITE','TELEFONE'] = None
data_teste.to_json("en_amostra_json/EN_db_PUBLIC_BASE_02_APPLE.json", orient="records")
data_teste['LATITUDE','LONGITUDE'] = None
data_teste.to_json("en_amostra_json/EN_db_PUBLIC_BASE_01_APPLE.json", orient="records")

In [11]:
dados.to_csv('amostra_json/BD_AMOSTRA_APPLE.csv', index=False)
data.to_csv('en_amostra_json/DB_SHOW_APPLE.csv', index=False)

In [12]:
# English version
data.to_json("en_amostra_json/EN_db_SIGA_05_APPLE.json", orient="records")
data['OPENING_HOURS','SERVICE_OPTIONS','INSTAGRAM','FACEBOOK'] = None
data.to_json("en_amostra_json/EN_db_SIGA_04_APPLE.json", orient="records")
data['CATEGORY_(CNAE)']  = None
data.to_json("en_amostra_json/EN_db_SIGA_03_APPLE.json", orient="records")
data['SITE','TELEFONE'] = None
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_02_APPLE.json", orient="records")
data['LATITUDE','LONGITUDE'] = None
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_01_APPLE.json", orient="records")

# Versão português
dados.to_json("amostra_json/PT_bd_SIGA_05_APPLE.json", orient='records')
dados['HORARIO_FUNCIONAMENTO','OPCOES_DE_SERVICO','INSTAGRAM','FACEBOOK'] = None
dados.to_json("amostra_json/PT_bd_SIGA_04_APPLE.json", orient='records')
dados['CNAE_DESCRICAO'] = None
dados.to_json("amostra_json/PT_bd_SIGA_03_APPLE.json", orient='records')
dados['SITE','TELEFONE'] = None
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_02_APPLE.json", orient="records")    
dados['LATITUDE', 'LONGITUDE'] = None
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_01_APPLE.json", orient="records")

In [11]:
# English version
data.to_json("en_amostra_json/EN_db_SIGA_05_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE','CATEGORY_(CNAE)']]
data.to_json("en_amostra_json/EN_db_SIGA_04_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE','PHONE_NUMBER', 'SITE']]  
data.to_json("en_amostra_json/EN_db_SIGA_03_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE']]
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_02_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE']]
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_01_APPLE.json", orient="records")

# Versão português
dados.to_json("amostra_json/PT_bd_SIGA_05_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE',
       'CNAE_DESCRICAO']]
dados.to_json("amostra_json/PT_bd_SIGA_04_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE']]
dados.to_json("amostra_json/PT_bd_SIGA_03_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE']]
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_02_APPLE.json", orient="records")    
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP']]
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_01_APPLE.json", orient="records")